In [24]:
import os, subprocess
import json
import uproot3
import awkward as ak
import numpy as np
from coffea import processor, util, hist
import pandas as pd
import pickle

from plotter import *

In [25]:
lumis = {}
lumis['2016'] = 35.9
lumis['2017'] = 41.5
lumis['2018'] = 59.9

with open('xsec.json') as f:
  xs = json.load(f)

with open('pmap.json') as f:
  pmap = json.load(f)

In [26]:
year = '2017'
nfiles = len(subprocess.getoutput("ls infiles-split/"+year+"*.json").split())
outsum = processor.dict_accumulator()

In [27]:
repickle=False

# Check if pickle exists, and don't re-create it if it does
picklename1 = 'pickles/'+year+'_muonkin.pkl'
if not os.path.isfile(picklename1):
    repickle=True
picklename2 = 'pickles/'+year+'_mujetkin.pkl'
if not os.path.isfile(picklename2):
    repickle=True

In [28]:
# Load all files - this takes a while
if repickle:
    nfiles = len(subprocess.getoutput("ls infiles-split/"+year+"*.json").split())
    for n in range(1,nfiles+1):

        with open('infiles-split/'+year+'_'+str(n)+'.json') as f:
            infiles = json.load(f)
    
        filename = '/myeosdir/inclusive-muon/outfiles/'+year+'_'+str(n)+'.coffea'
        if os.path.isfile(filename):
            out = util.load(filename)
            outsum.add(out)
        else:
            print('Missing file '+str(n),infiles.keys())
            #print("File " + filename + " is missing")
        
    scale_lumi = {k: xs[k] * 1000 *lumis[year] / w for k, w in outsum['sumw'].items()}

    muonkin = outsum['muonkin']
    mujetkin = outsum['mujetkin']
    
    muonkin.scale(scale_lumi, 'dataset')
    mujetkin.scale(scale_lumi, 'dataset')

    muonkin = muonkin.group('dataset', hist.Cat('process', 'Process'), pmap)
    mujetkin = mujetkin.group('dataset', hist.Cat('process', 'Process'), pmap)

    outfile = open(picklename1, 'wb')
    pickle.dump(muonkin, outfile, protocol=-1)
    outfile.close()
    
    outfile = open(picklename2, 'wb')
    pickle.dump(mujetkin, outfile, protocol=-1)
    outfile.close()

In [29]:
cutflow = pickle.load(open('pickles/'+str(year)+'_cutflow.pkl','rb'))
cutflow = cutflow.sum('genflavor').integrate('region','muoncontrol')

In [30]:
cutflow = outsum['cutflow'].group('dataset', hist.Cat('process', 'Process'), pmap).sum('genflavor').integrate('region','muoncontrol')

KeyError: 'cutflow'

In [ ]:
cutflow.integrate('process','muondata').values()

In [ ]:
cutflow.integrate('process','ttbar').values()

In [ ]:
df1 = pd.DataFrame([])

df1['QCD'] = cutflow.values()[('QCD',)]
df1['Wjets'] = cutflow.values()[('Wjets',)]
df1['Zjets'] = cutflow.values()[('Zjets',)]
df1['VV'] = cutflow.values()[('VV',)]
df1['ttbar'] = cutflow.values()[('ttbar',)]
df1['singlet'] = cutflow.values()[('singlet',)]
df1['data'] = cutflow.values()[('muondata',)]

df1 = df1[1:9].astype('int')
df1.index = ['Jet kinematics','Jet ID','Jet $N_2^\text{DDT}$','b-tag','No e/tau','One muon','Dphi(muon,AK8)','DeepDoubleB']
df1.to_latex(buf=year+'/cutflow-muoncr.tex')

In [ ]:
templates = pickle.load(open('pickles/'+str(year)+'_templates.pkl','rb'))
templates = templates.integrate('systematic','nominal')
muonkin = pickle.load(open('pickles/'+str(year)+'_muonkin.pkl','rb'))
mujetkin = pickle.load(open('pickles/'+str(year)+'_mujetkin.pkl','rb'))

In [ ]:
h = templates.integrate('region', 'muoncontrol').sum('pt1').integrate('ddb1',int_range=slice(0,0.89))
plot_datamc_muoncr(h,year+'/muCR_msd1_fail',year+" muon CR, DDB fail")
h = templates.integrate('region', 'muoncontrol').sum('pt1').integrate('ddb1',int_range=slice(0.89,1))
plot_datamc_muoncr(h,year+'/muCR_msd1_pass',year+" muon CR, DDB pass")
h = templates.integrate('region', 'muoncontrol').sum('ddb1','pt1')
plot_datamc_muoncr(h,year+'/muCR_msd1',year+" muon CR")

In [ ]:
h = templates.integrate('region', 'muoncontrol').sum('ddb1','msd1')
plot_datamc_muoncr(h,year+'/muCR_pt1',year+" muon CR")

In [ ]:
h = mujetkin.integrate('region', 'muoncontrol').sum('ddb1')
plot_datamc_muoncr(h,year+'/muCR_eta1',year+" muon CR")

In [ ]:
h = mujetkin.integrate('region', 'muoncontrol').sum('eta1')
plot_datamc_muoncr(h,year+'/muCR_ddb1',year+" muon CR")

In [ ]:
h = muonkin.integrate('region', 'muoncontrol').sum('etamu').integrate('ddb1',int_range=slice(0.89,1))
plot_datamc_muoncr(h,year+'/muCR_ptmu_pass',year+" muon CR, DDB pass")
h = muonkin.integrate('region', 'muoncontrol').sum('etamu').integrate('ddb1',int_range=slice(0,0.89))
plot_datamc_muoncr(h,year+'/muCR_ptmu_fail',year+" muon CR, DDB fail")
h = muonkin.integrate('region', 'muoncontrol').sum('etamu','ddb1')
plot_datamc_muoncr(h,year+'/muCR_ptmu',year+" muon CR")

In [ ]:
h = muonkin.integrate('region', 'muoncontrol').sum('ptmu').integrate('ddb1',int_range=slice(0.89,1))
plot_datamc_muoncr(h,year+'/muCR_etamu_pass',year+" muon CR, DDB pass")
h = muonkin.integrate('region', 'muoncontrol').sum('ptmu').integrate('ddb1',int_range=slice(0,0.89))
plot_datamc_muoncr(h,year+'/muCR_etamu_fail',year+" muon CR, DDB fail")
h = muonkin.integrate('region', 'muoncontrol').sum('ptmu','ddb1')
plot_datamc_muoncr(h,year+'/muCR_etamu',year+" muon CR")